<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jaiio/00-04-static_features_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de las features adicionales a la Consulta


## Features estáticas a partir de los datos

1. Traigo los datos y los cargo en el dataframe:

In [62]:
from os import path
import pandas as pd

# Traigo el archivo desde GitHub
!wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/50jaiio/consolidados/feature-extraction/dataset-ss3.csv

df = pd.read_csv('dataset-ss3.csv')

df['clase'].value_counts()


--2021-06-20 00:18:30--  https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/50jaiio/consolidados/feature-extraction/dataset-ss3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423523 (414K) [text/plain]
Saving to: ‘dataset-ss3.csv’

dataset-ss3.csv     100%[===================>] 413.60K  --.-KB/s    in 0.06s   

2021-06-20 00:18:30 (6.39 MB/s) - ‘dataset-ss3.csv’ saved [423523/423523]



Exámenes                               50
Datos Personales                       50
Consulta sobre Título Universitario    50
Pedido de Certificados                 50
Requisitos de Ingreso                  50
Cursadas                               50
Ingreso a la Universidad               50
Consulta por Legajo                    50
Reincorporación                        50
Simultaneidad de Carreras              50
Problemas con la Clave                 50
Cambio de Carrera                      50
Situación Académica                    50
Cambio de Comisión                     50
Boleto Universitario                   50
Consulta por Equivalencias             50
Name: clase, dtype: int64

In [63]:
df.columns

Index(['fecha', 'hora', 'apellido_nombre', 'legajo', 'documento', 'carrera',
       'telefono', 'email', 'consulta', 'respuesta', 'score', 'clase'],
      dtype='object')

2. Verifico los datos y borro columnas no pre-procesables:

In [64]:
df.drop(columns=['apellido_nombre', 'respuesta'], inplace=True)
df.head()

,fecha,hora,legajo,documento,carrera,telefono,email,consulta,score,clase
0,10-15-2019,04:06:31,160474,40993972,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",36.964314,Boleto Universitario
1,08-06-2018,10:59:04,153748,34368522,contador publico(54),NaN,nemercurio@gmail.com,quiero acceder al beneficio de boleto estudian...,36.109413,Boleto Universitario
2,07-30-2018,20:20:20,165259,42344375,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,35.828820,Boleto Universitario
3,04-09-2019,18:28:42,146293,39749232,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario
4,04-09-2019,18:28:42,146293,39749232,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com <mailto:boscomatias@ho...,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario


3. Preproceso la fecha (día de la semana, semana del mes, mes y cuatrimestre) de la consulta:

In [65]:
def convierte_fecha(fecha_consulta):
  from datetime import datetime
  # Convierto a fecha
  fecha = datetime.strptime(fecha_consulta, '%m-%d-%Y') 
  return fecha

def dia_semana(fecha):
  # Tomo el día de la semana
  dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
  # dia_semana = dias_semana[fecha.weekday()]
  dia_semana = fecha.weekday()
  return dia_semana

def semana_del_mes(fecha):
  # Tomo la semana del mes
  semana_mes = (fecha.day-1)//7+1
  return semana_mes

def mes(fecha):
  # Tomo el mes
  meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
  # mes = meses[fecha.month-1]
  mes = fecha.month
  return mes

def cuatrimestre(fecha):
  # Defino el cuatrimestre
  cuatrimestre = 1
  if (fecha.month>7):
    cuatrimestre = 2
  return cuatrimestre

def anio(fecha):
    anio = fecha.year
    return anio

# convierte_fecha(df.loc[0, 'Fecha'])
fecha = df['fecha'].apply(convierte_fecha)
df['dia_semana'] = fecha.apply(dia_semana)
df['semana_del_mes'] = fecha.apply(semana_del_mes)
df['mes'] = fecha.apply(mes)
df['cuatrimestre'] = fecha.apply(cuatrimestre)
df['anio'] = fecha.apply(anio)

In [66]:
df.drop(columns=['fecha'], inplace=True)
df.head()

,hora,legajo,documento,carrera,telefono,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio
0,04:06:31,160474,40993972,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",36.964314,Boleto Universitario,1,3,10,2,2019
1,10:59:04,153748,34368522,contador publico(54),NaN,nemercurio@gmail.com,quiero acceder al beneficio de boleto estudian...,36.109413,Boleto Universitario,0,1,8,2,2018
2,20:20:20,165259,42344375,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,35.828820,Boleto Universitario,0,5,7,1,2018
3,18:28:42,146293,39749232,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019
4,18:28:42,146293,39749232,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com <mailto:boscomatias@ho...,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019


4. Preproceso la hora de la consulta de la siguiente forma:
    - 0-Mañana (6-12 hs),
    - 1-Media-Tarde (12-16 hs)
    - 2-Tarde (16-20 hs)
    - 3-Noche (20-00 hs)
    - 4-Madrugada (00-6 hs)

In [67]:
def convierte_horario(hora_consulta):
  from datetime import datetime
  # Convierto a hora
  hora = datetime.strptime(hora_consulta, '%H:%M:%S').time()
  return hora

def discretiza_horario(horario):
  if (horario.hour>=6 and horario.hour<12):
    rango_horario=0
  elif (horario.hour>=12 and horario.hour<16):
    rango_horario=1
  elif (horario.hour>=16 and horario.hour<20):
    rango_horario=2
  else:
    rango_horario=3 
  return rango_horario

#discretiza_horario(convierte_horario(df.loc[0, 'Hora']))
horario = df['hora'].apply(convierte_horario)
df['hora_discretizada'] = horario.apply(discretiza_horario)

In [68]:
df.drop(columns=['hora'], inplace=True)
df.head()

,legajo,documento,carrera,telefono,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada
0,160474,40993972,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",36.964314,Boleto Universitario,1,3,10,2,2019,3
1,153748,34368522,contador publico(54),NaN,nemercurio@gmail.com,quiero acceder al beneficio de boleto estudian...,36.109413,Boleto Universitario,0,1,8,2,2018,0
2,165259,42344375,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,35.828820,Boleto Universitario,0,5,7,1,2018,3
3,146293,39749232,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2
4,146293,39749232,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com <mailto:boscomatias@ho...,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2


5. Preproceso el legajo y el DNI:<br />
  Los separo en cuantiles asumiendo que los DNI mas bajos son personas mas grandes y los legajos mas grandes son estudiantes mas nuevos.

In [69]:
import numpy as np
 
def discretiza_atributo(atributo, intervalos=4):
  # Se discretiza según la cantidad de q definidos
  discretizado = pd.qcut(atributo, q=intervalos)
  
  # Nos quedamos con los diferentes intervalos, los ordenamos min-max
  lista_intervalos = discretizado.unique()

  lista_intervalos.sort_values(inplace=True)
  #  Lo casteamos a str
  lista_intervalos = lista_intervalos.astype(str)
  #  Casteamos el atributo a str
  discretizado = discretizado.astype(str)
  # Borro los nan porque los modifico a mano
  tiene_nan = False
  if ('nan' in lista_intervalos):
    tiene_nan = True
    np.delete(lista_intervalos, np.where(lista_intervalos == 'nan'))

  for i in range(intervalos):
    # Si existiera, quiero dejar el nan en el intervalo con etiqueta 0, por eso sumo 1 a i
    discretizado = discretizado.replace(lista_intervalos[i], i+1)
    discretizado = discretizado.replace('nan', 0)

  return discretizado

# Cambio a numericos los Documentos y los discretizo
df["documento"] = pd.to_numeric(df["documento"], errors='coerce')
df["dni_discretizado"] = discretiza_atributo(df["documento"], 8)

# Cambio a numericos los Legajos y los discretizo
df["legajo"] = pd.to_numeric(df["legajo"], errors='coerce')
df["legajo_discretizado"] = discretiza_atributo(df["legajo"], 4)

In [70]:
df["dni_discretizado"].value_counts()

8    100
7    100
5    100
3    100
2    100
1    100
6     99
4     99
0      2
Name: dni_discretizado, dtype: int64

In [71]:
df["legajo_discretizado"].value_counts()

0    211
1    148
4    147
3    147
2    147
Name: legajo_discretizado, dtype: int64

6. Genero una variable dummy con la existencia o no de Legajo y Teléfono:

In [72]:
# Tiene legajo?
def posee_valor(atributo):
  return 1-atributo.isna()
  
df['posee_legajo'] = posee_valor(df['legajo'])

In [73]:
df.drop(columns=['legajo', 'documento'], inplace=True)
df.head()

,carrera,telefono,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo
0,licenciatura en administracion(3),NaN,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",36.964314,Boleto Universitario,1,3,10,2,2019,3,7,4,1
1,contador publico(54),NaN,nemercurio@gmail.com,quiero acceder al beneficio de boleto estudian...,36.109413,Boleto Universitario,0,1,8,2,2018,0,4,4,1
2,licenciatura en administracion(3),3489441201,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,35.828820,Boleto Universitario,0,5,7,1,2018,3,8,4,1
3,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1
4,licenciatura en administracion(3),1140878148,boscomatias@hotmail.com <mailto:boscomatias@ho...,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1


In [74]:
# Tiene teléfono? 
df['posee_telefono'] = posee_valor(df['telefono'])

In [75]:
df.drop(columns=['telefono'], inplace=True)
df.head()

,carrera,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono
0,licenciatura en administracion(3),mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",36.964314,Boleto Universitario,1,3,10,2,2019,3,7,4,1,0
1,contador publico(54),nemercurio@gmail.com,quiero acceder al beneficio de boleto estudian...,36.109413,Boleto Universitario,0,1,8,2,2018,0,4,4,1,0
2,licenciatura en administracion(3),agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,35.828820,Boleto Universitario,0,5,7,1,2018,3,8,4,1,1
3,licenciatura en administracion(3),boscomatias@hotmail.com,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1,1
4,licenciatura en administracion(3),boscomatias@hotmail.com <mailto:boscomatias@ho...,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1,1


7. Se extrae el código de Carrera:

In [76]:
def extrae_codigo_carrera(texto_carrera):
  codigo = 0
  if "sin carrera" not in texto_carrera:
    texto_carrera = texto_carrera.split('(')
    texto_carrera = texto_carrera[len(texto_carrera)-1].split(')')
    codigo = int(texto_carrera[0])
  return codigo

# extrae_codigo_carrera(df.loc[0, 'Carrera'])
df['carrera_valor'] = df['carrera'].apply(extrae_codigo_carrera)

In [77]:
df.drop(columns=['carrera'], inplace=True)
df.head()

,email,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor
0,mancubruno@gmail.com,"hola, en la página dice que tengo asignado el ...",36.964314,Boleto Universitario,1,3,10,2,2019,3,7,4,1,0,3
1,nemercurio@gmail.com,quiero acceder al beneficio de boleto estudian...,36.109413,Boleto Universitario,0,1,8,2,2018,0,4,4,1,0,54
2,agustinvandick@gmail.com,yo hace casi un mes hice el tramite para la su...,35.828820,Boleto Universitario,0,5,7,1,2018,3,8,4,1,1,3
3,boscomatias@hotmail.com,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1,1,3
4,boscomatias@hotmail.com <mailto:boscomatias@ho...,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1,1,3


8. Se verifica el proveedor de correo electrónico:

In [78]:
def servicio_email(consulta, proveedor):  
  if consulta.lower().find(proveedor)==-1:
    return 0
  else:
    return 1
# Correo Gmail?
# df['correo_gmail'] = df['E-mail'].apply(servicio_email, proveedor='gmail')
# Correo yahoo?
# df['correo_yahoo'] = df['E-mail'].apply(servicio_email, proveedor='yahoo')
# Correo hotmail?
# df['correo_hotmail'] = df['E-mail'].apply(servicio_email, proveedor='hotmail')

def extrae_proveedor_correo(texto_correo):
    texto_correo = texto_correo.split('@')
    texto_correo = texto_correo[len(texto_correo)-1].split('.')
    proveedor_correo = texto_correo[0]
    return proveedor_correo

#extrae_proveedor_correo(df.loc[0, 'E-mail'])
df['proveedor_correo'] = df['email'].apply(extrae_proveedor_correo)

In [79]:
df['proveedor_correo'].value_counts()

hotmail                 388
gmail                   298
yahoo                    50
outlook                  37
live                     17
icloud                    2
gmx                       2
correoargentino           1
fangioconcesionarios      1
dnpdistribuciones         1
hotmaill                  1
udesa                     1
osde                      1
Name: proveedor_correo, dtype: int64

In [80]:
# Limpio el campo proveedor_correo

df['proveedor_correo'].replace({"autloock": "outlook",
                                "outlool": "outlook",
                                "gamail": "gmail",
                                "gmal": "gmail",
                                "gmil": "gmail",
                                "hotmaill": "hotmail",
                                "luve": "live",
                                }, inplace=True)

# Se transforma proveedor_correo a numerico
from sklearn import preprocessing
le_correo = preprocessing.LabelEncoder()
df['proveedor_correo'] = le_correo.fit_transform(df['proveedor_correo'])

le_correo.classes_

array(['correoargentino', 'dnpdistribuciones', 'fangioconcesionarios',
       'gmail', 'gmx', 'hotmail', 'icloud', 'live', 'osde', 'outlook',
       'udesa', 'yahoo'], dtype=object)

In [81]:
df.drop(columns=['email'], inplace=True)
df.head()

,consulta,score,clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo
0,"hola, en la página dice que tengo asignado el ...",36.964314,Boleto Universitario,1,3,10,2,2019,3,7,4,1,0,3,3
1,quiero acceder al beneficio de boleto estudian...,36.109413,Boleto Universitario,0,1,8,2,2018,0,4,4,1,0,54,3
2,yo hace casi un mes hice el tramite para la su...,35.828820,Boleto Universitario,0,5,7,1,2018,3,8,4,1,1,3,3
3,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1,1,3,5
4,hola queria saber porque no se me acredita la ...,32.916737,Boleto Universitario,1,2,4,1,2019,2,6,3,1,1,3,5


## Features estáticas léxicas

Genero las features estáticas léxicas basadas en caracteres:

In [82]:
# Total number of characters
def cantidad_caracteres(columna_consulta):
  return columna_consulta.str.len()

df['cantidad_caracteres'] = cantidad_caracteres(df.consulta)

# Proporcion de mayúsculas en la consulta
def proporcion_mayusculas(consulta):
  return sum(1 for letra in consulta if letra.isupper())

df['proporcion_mayusculas'] = df['consulta'].apply(proporcion_mayusculas)/df['cantidad_caracteres']

# Proporción de letras en la consulta
def proporcion_letras(consulta):
  return sum(1 for letra in consulta if letra.isalpha())

df['proporcion_letras'] = df['consulta'].apply(proporcion_letras)/df['cantidad_caracteres']

# Cantidad de letras con tildes
def cantidad_tildes(consulta):
  return sum(1 for letra in consulta if letra.lower() in ['á', 'é', 'í', 'ó', 'ú'])

df['cantidad_tildes'] = df['consulta'].apply(cantidad_tildes)

Genero las features estáticas léxicas basadas en palabras:

In [83]:
# total number of words
def cantidad_palabras(consulta):
  palabras = consulta.split(sep=' ')
  return len(palabras)

df['cantidad_palabras'] = df['consulta'].apply(cantidad_palabras)

# proportion of short words (less than four characters)
def cantidad_palabras_cortas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return sum(1 for palabra in palabras if len(palabra) <= letras_corta)

df['cantidad_palabras_cortas'] = df['consulta'].apply(cantidad_palabras_cortas)

# ratio of number of distinct words to the total number of words: |set(words)|/|words|
def cantidad_palabras_distintas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return len(set(palabras))

df['proporcion_palabras_distintas'] = df['consulta'].apply(cantidad_palabras_distintas)/df['cantidad_palabras']
df['proporcion_palabras_distintas']

0      0.885714
1      0.833333
2      0.785714
3      0.703704
4      0.703704
         ...   
795    0.782609
796    0.866667
797    0.956522
798    0.878049
799    0.869565
Name: proporcion_palabras_distintas, Length: 800, dtype: float64

Genero las features estáticas sintácticas:

In [84]:
# Frecuencia de signos de puntuación {,.¿?!:;’"}
def cantidad_signos_puntuacion(consulta):
  signos_puntuacion = [',', '.', '¿', '?', '!', '¡', ':', ';', '"']
  return sum(1 for letra in consulta if letra.lower() in signos_puntuacion)

df['frecuencia_signos_puntuacion'] = df['consulta'].apply(cantidad_signos_puntuacion)/df['cantidad_caracteres']

Genero las features estáticas estructurales:

In [85]:
# total number of sentences
def cantidad_oraciones(consulta):
  if consulta.count('.')==0:
    return 1
  else: 
    return consulta.count('.')

df['cantidad_oraciones'] = df['consulta'].apply(cantidad_oraciones)

# utiliza código de asignatura?
def utiliza_codigo_asignatura(consulta):
  codigo=0
  palabras = consulta.split(sep=' ')
  for palabra in palabras:
    if palabra.isdigit():
      if int(palabra)>=10000 and int(palabra)<=99999:
        codigo=1
  return codigo

df['utiliza_codigo_asignatura'] = df['consulta'].apply(utiliza_codigo_asignatura)

In [86]:
len(df.columns)

25

Por último, reordeno el score y la clase para que me quede última:

In [87]:
x = df['score']
y = df['clase']
df.drop(columns=['clase', 'score'], inplace=True)
df.insert(len(df.columns), "score", x, True)
df.insert(len(df.columns), "clase", y, True)

In [88]:
# Tomo una consulta determinada
df.loc[1, 'consulta']

# Verifico las columnas del Dataframe
df.head()

,consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,score,clase
0,"hola, en la página dice que tengo asignado el ...",1,3,10,2,2019,3,7,4,1,0,3,3,207,0.0,0.816425,2,35,17,0.885714,0.019324,1,0,36.964314,Boleto Universitario
1,quiero acceder al beneficio de boleto estudian...,0,1,8,2,2018,0,4,4,1,0,54,3,229,0.0,0.803493,0,42,23,0.833333,0.017467,1,0,36.109413,Boleto Universitario
2,yo hace casi un mes hice el tramite para la su...,0,5,7,1,2018,3,8,4,1,1,3,3,299,0.0,0.799331,0,56,33,0.785714,0.016722,3,0,35.828820,Boleto Universitario
3,hola queria saber porque no se me acredita la ...,1,2,4,1,2019,2,6,3,1,1,3,5,253,0.0,0.762846,0,54,34,0.703704,0.027668,3,0,32.916737,Boleto Universitario
4,hola queria saber porque no se me acredita la ...,1,2,4,1,2019,2,6,3,1,1,3,5,253,0.0,0.762846,0,54,34,0.703704,0.027668,3,0,32.916737,Boleto Universitario


10. Guardo el csv procesado:

In [90]:
from google.colab import drive
drive.mount('drive')

df.to_csv('dataset-ss3-prep.csv', index=False)

!cp dataset-ss3-prep.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
